## Starting with scraping all the player stats for the 2022 Orix Buffalos

In [170]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import pandas as pd
import re 

In [168]:
def get_stats_table(url: str, id: str):  
    """
    Scrapes the pitching and batting table depending on the ID provided
    Pitching table is in a comment for some reason so have to do some 
    cleaning before converting to DF
    """
    
    s = HTMLSession()
    page = s.get(url)
    stats_div = BeautifulSoup(page.content, "html.parser")

    if id == "team_pitching":
        s = str(stats_div.find("div", id="all_team_pitching"))
        start_len, end_len = s.find('<table'), s.rfind('</table>')
        cleaned_page = s[start_len:end_len + len('</table>')]
        stats_div = BeautifulSoup(cleaned_page, "html.parser")  
        
    stats_table = stats_div.find("table", id=id)

    stats_df = pd.DataFrame()
    for row in stats_table.tbody.find_all("tr"):
        columns = row.find_all("td")
        
        if(columns != []):
            stats_dict = {}
            for i in range(0, len(columns)):
                stats_dict[columns[i]["data-stat"]] = columns[i].text.strip()
            stats_df = stats_df.append(stats_dict, ignore_index=True)

    return stats_df

In [ ]:
url = "https://www.baseball-reference.com/register/team.cgi?id=d5c0c304"

batting_stats = get_stats_table(url, "team_batting")
pitching_stats = get_stats_table(url, "team_pitching")
